In [3]:
#알라딘 - 제목, 저자, 장르, isbn // 1시간 1만회 총 10만건 하루

import requests
import json
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from tqdm import tqdm

알라딘 정보 가져오기

In [3]:
def make_data(Year, Month, Week = 1):
    TTBKey = "ttbleejunho09091938001"
    df = pd.DataFrame({
        'title' : [],
        'author' : [],
        'genre_id' : [], #알라딘 category id
        'genre' : [],
        'ISBN' : [],
        'description_aladin' : [], #알라딘 상세정보
        'description_naver' : [], #네이버 상세정보
        'salesPoint': [], #알라딘 판매지수 (인기도)
        'mallType': [] #종류
    })
    #start : 1에서 20까지 총 1000권만 받을 수 있고 0과 21이상의 값이 들어가면 1과 같은 값들만 나옴.
    #->베스트셀러 데이터를 월별로 만들고 중복되는걸 없애면 될 듯.
    
    for i in range(1, 21):
        url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={TTBKey}&QueryType=Bestseller&Year={Year}&Month={Month}&Week={Week}&MaxResults=50&start={i}&SearchTarget=Book&output=JS&Version=20131101"
        res = requests.get(url)
        res.raise_for_status()
    
        for item in res.json()['item']:
            title = item['title']
            author = item['author']
            genre_id = item['categoryId']  
            genre = item['categoryName']
            isbn = item['isbn13']
            description_aladin = item['description']
            description_naver = ""
            salesPoint = item['salesPoint']
            mallType = item['mallType']
            
            res = [title, author, genre_id, genre, isbn, description_aladin, description_naver, salesPoint, mallType]
            df.loc[len(df)] = res
    
    with open(f'알라딘 베스트 top 1000 - {Year}_{Month}.csv', 'w', encoding='utf-8-sig') as f:
        df.to_csv(f, index=False)
    f.close()

In [34]:
import time
#2000~2024년까지데이터 만들기
for y in range(2000, 2024):
    for m in range(1,13):
        if m in (1,4,7,9):
            make_data(y,m)
            time.sleep(10)


In [4]:
make_data(2015,1)

알라딘 데이터에 네이버 상세설명 추가

In [4]:
#네이버 - 책 소개 // 일 허용량 25000회
#네이버 API 활용을 위한 ID, 비밀번호

def add_description_naver(path):
    #df = pd.read_csv(path, encoding='utf-8-sig')
    df = pd.read_csv(path, encoding='utf-8-sig', dtype={'ISBN' : 'str', 'description_naver' : 'str'})
    df['ISBN'] = df['ISBN'].str.replace(r'\..*', '', regex=True)
    #df['ISBN'] = df['ISBN'].fillna('0')
    
    client_ID = "uJUq5VpE1of93TclleEn"
    client_Secret = "sbkTUgIxL6"
    
    for i in tqdm(range(0,len(df))):
        if pd.isnull(df['description_naver'].iloc[i]) and not pd.isnull(df['ISBN'].iloc[i]):
            query = df['ISBN'].iloc[i]
            encoded_query = urllib.parse.quote(query) #utf-8로 인코딩
            
            url = f"https://openapi.naver.com/v1/search/book_adv.xml?d_isbn={encoded_query}&display=10&start=1"
            headers = {
                "X-Naver-Client-Id": client_ID,
                "X-Naver-Client-Secret": client_Secret
            }
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.content, 'xml')
            descriptions = soup.find_all('description')
            
            if len(descriptions) == 2:
                description = descriptions[1].text.strip()
                df['description_naver'].iloc[i] = description
               

    df.to_csv(path, encoding='utf-8-sig', index=False)

ISBN데이터 있는데도 상세설명 추가 안되는 것들 이유 알아보기
: 판매중지된 상품, 특별판/기념판/묶음상품/시즌상품 검색 안됨
: 진짜 없는거도 있음
스스로 행복하라 (법정 스님 열반 10주기 특별판, 샘터 50주년 지령 600호 기념판)
2020 시나공 정보처리기사 필기 (NCS  기반 전면 개편)

In [3]:
test = pd.read_csv("2020-2024/알라딘 베스트 top 1000 - 2020_4.csv", encoding='utf-8-sig', dtype = {'ISBN' : 'str'})
test2 = test[test['description_naver'].isnull()]['ISBN']
test2

6      9788932920184
43     9791196719432
59     9791164421114
63     9791163718970
92     9791163718963
           ...      
918    9791165340919
935    9791189345907
970    9791163716907
989    9791159629198
991    9788969651662
Name: ISBN, Length: 86, dtype: object

In [127]:
client_ID = "uJUq5VpE1of93TclleEn"
client_Secret = "sbkTUgIxL6"
for isbn in test2:
    url = f"https://openapi.naver.com/v1/search/book_adv.xml?d_isbn={isbn}&display=10&start=1"
    
    headers = {
        "X-Naver-Client-Id": client_ID,
        "X-Naver-Client-Secret": client_Secret
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'xml')
    print(soup)

<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0"><channel><title>Naver Open API - book_adv ::''</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Wed, 18 Sep 2024 11:35:47 +0900</lastBuildDate><total>0</total><start>1</start><display>0</display></channel></rss>
<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0"><channel><title>Naver Open API - book_adv ::''</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Wed, 18 Sep 2024 11:35:48 +0900</lastBuildDate><total>0</total><start>1</start><display>0</display></channel></rss>
<?xml version="1.0" encoding="utf-8"?>
<rss version="2.0"><channel><title>Naver Open API - book_adv ::''</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Wed, 18 Sep 2024 11:35:48 +0900</lastBuildDate><total>1</total><start>1</start><display>1</display><item><title>스스로 행복하라 (법정 스님 열반 10주기 특별

KeyboardInterrupt: 

추가하기

In [ ]:
import os

#folder_path안 year 연도의 naver 상세정보 저장
folder_path = "2005-2009"
year = 2006
file_paths = []
pd.options.mode.chained_assignment = None
for filename in os.listdir(folder_path):
    if filename.endswith('.csv') and str(year) in filename:
        add_description_naver(os.path.join(folder_path, filename))


  1%|▉                                                                               | 11/1000 [00:15<26:29,  1.61s/it]

In [4]:
import os
folder_path = "2005-2009"
pd.options.mode.chained_assignment = None
list = ["알라딘 베스트 top 1000 - 2011_7.csv", "알라딘 베스트 top 1000 - 2011_9.csv", "알라딘 베스트 top 1000 - 2012_7.csv",
       "알라딘 베스트 top 1000 - 2012_9.csv"] 
for i in list:
    add_description_naver(os.path.join(folder_path, i))

100%|████████████████████████████████████████████████████████████████████████████████| 808/808 [08:20<00:00,  1.61it/s]


In [5]:
pd.options.mode.chained_assignment = None
add_description_naver("2020-2024/알라딘 베스트 top 1000 - 2020_9.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [20:35<00:00,  1.24s/it]


알라딘 베스트 데이터 날짜 간격 기준 설정
한달 간격 : 중복 약 50%
두 달 간격 : 중복 약 35%
세 달 간격 : 중복 약 30%
-> 편하게 그냥 분기별로

In [4]:
test1 = pd.read_csv("알라딘 베스트 top 1000 - 2024_1_1.csv",encoding='utf-8-sig')
test2 = pd.read_csv("알라딘 베스트 top 1000 - 2024_2_1.csv",encoding='utf-8-sig')
test3 = pd.read_csv("알라딘 베스트 top 1000 - 2024_3_1.csv",encoding='utf-8-sig')
test4 = pd.read_csv("알라딘 베스트 top 1000 - 2024_4_1.csv",encoding='utf-8-sig')
test5 = pd.read_csv("알라딘 베스트 top 1000 - 2024_5_1.csv",encoding='utf-8-sig')
test6 = pd.read_csv("알라딘 베스트 top 1000 - 2024_6_1.csv",encoding='utf-8-sig')
test7 = pd.read_csv("알라딘 베스트 top 1000 - 2024_7_1.csv",encoding='utf-8-sig')
test8 = pd.read_csv("알라딘 베스트 top 1000 - 2024_8_1.csv",encoding='utf-8-sig')

In [7]:
print(test1.duplicated().sum(), test2.duplicated().sum())
print(len(test1), len(test2), len(test3), len(test4), len(test5), len(test6), len(test7), len(test8))

0 0
1000 1000 1000 1000 1000 1000 1000 1000


In [21]:
dup_1_2 = pd.concat([test1, test2], axis=0)
dup_1_3 = pd.concat([test1, test3], axis=0)
dup_1_4 = pd.concat([test1, test4], axis=0)
dup_1_5 = pd.concat([test1, test5], axis=0)
dup_1_6 = pd.concat([test1, test6], axis=0)
dup_1_7 = pd.concat([test1, test7], axis=0)
dup_1_8 = pd.concat([test1, test8], axis=0)

dup_1_2_3 = pd.concat([dup_1_2, test3], axis=0)

In [22]:
for data in (dup_1_2, dup_1_3, dup_1_4,dup_1_5,dup_1_6, dup_1_7, dup_1_8, dup_1_2_3):
    print(data.duplicated().sum())

514
376
309
263
239
224
214
1072
